# Merge and group tables from MS Excel

One of the most used formulas in MS Excel to merge (enrich) data from different sources is vlookup. However the amount of cells with vlookup formulas is limited due to both the peformance of hardware and MS Excel. When data gets bigger, waiting times rise and there are freeze times and software crashes on an average work place equipment. As a reaction the amount of data is usually limited which causes multiple preparation times and additional steps.

To analyise more data (and get holistic and cross-divisional results and findings) but still use workplace equipment and remain independent from other IT resources, vlookups can be done via python. Once prepared, the data can be handled in Excel/PowerQuery/Powerpivot).
Remark: High volume data analysis should be handled in adequate hardware and there has to be some automation. Doing this regularly in MS Excel is not appropriate in terms of modern business management.

**Problem**: The data file from the general ledger of an accounting system has 129.000 entries. There are several years, months and versions and this export just names the cost account but the cost account name and its position within the chart of accounts is needed in connection with the booking data. Due to the nature of accounting, there are more than one booking on a cost account within a year, month, version and cost center. The data has be grouped on the level of cost accounts.

**Approach**: 

1. Import data from MS Excel to a pandas dataframe
2. Merge (enrich) data (as 'vlookup' in MS Excel, as 'join' in SQL)
3. Group data
4. Export data from dataframe to a .csv file

In [40]:
import pandas as pd
import numpy as np

#### 1. Import the data

In [41]:
# Import the accounting data from general ledger
# We know, there is header data in line 0, no endex column and the MS Excel sheet name is "Data"
df_data = pd.read_excel('01 DATA/DATA.XLSX', index_col=None, header=0, sheet_name='DATA')

In [42]:
# Import the chart of accounts
# We know, there is header data in line 0, no endex column and the MS Excel sheet name is "COA"
df_coa = pd.read_excel('02 DATA STRUCTURE/COA.XLSX', index_col=None, header=0, sheet_name='COA')

In [43]:
# Let´s take a look at the data
df_data.head(10)

,COMPANY CODE,FISCAL YEAR,MONTH,YEAR-MONTH,COST CENTER,GL ACCOUNT,VALUE,VERSION
0,ENTITY1,2018,1,2018-1,CCAAA460001,1210,14699.67,BUD
1,ENTITY1,2018,1,2018-1,CCAAA460001,2260,75719.84,BUD
2,ENTITY1,2018,1,2018-1,CCAAA460601,1210,575.44,BUD
3,ENTITY1,2018,1,2018-1,CCAAA460601,2260,2964.15,BUD
4,ENTITY1,2018,1,2018-1,CCAAA601101,2420,121570.05,BUD
5,ENTITY1,2018,1,2018-1,CCAAA601101,2430,8159.73,BUD
6,ENTITY1,2018,1,2018-1,CCAAA601101,2570,15698.42,BUD
7,ENTITY1,2018,1,2018-1,CCAAA601101,2620,9539.21,BUD
8,ENTITY1,2018,1,2018-1,CCAAA601101,2630,6338.13,BUD
9,ENTITY1,2018,1,2018-1,CCAAA601101,2650,7374.96,BUD


In [44]:
# Let´s take a look at the chart of accounts
df_coa.head(10)

,GL ACCOUNT,GL ACCOUNT NAME,FINANCIAL STATEMENT,GROUP,SUB-GROUP,DEBITCREDIT
0,100,Bank checking account,Balance sheet,Current assets,Cash and cash equivalents,Debit
1,110,Bank savings account,Balance sheet,Current assets,Cash and cash equivalents,Debit
2,120,Online savings account,Balance sheet,Current assets,Cash and cash equivalents,Debit
3,130,Petty cash account,Balance sheet,Current assets,Cash and cash equivalents,Debit
4,140,Paypal account,Balance sheet,Current assets,Cash and cash equivalents,Debit
5,200,Short term marketable securities,Balance sheet,Current assets,Short term marketable securities,Debit
6,300,Accounts receivable,Balance sheet,Current assets,Accounts receivable,Debit
7,310,Allowance for doubtful debts account,Balance sheet,Current assets,Accounts receivable,Credit
8,400,Raw materials,Balance sheet,Current assets,Inventory,Debit
9,410,Work in progress,Balance sheet,Current assets,Inventory,Debit


#### 2. Merge data

In [45]:
# We like to enrich the data table with all data from the coa table
# We want to enrich all lines of the data table
# This is a left join like in sql with parameters
# df_data is left
# df_coa is right

data_vlookup=pd.merge(df_data [['COMPANY CODE', 'FISCAL YEAR', 'MONTH', 'YEAR-MONTH', 'COST CENTER', 'GL ACCOUNT', 'VALUE', 'VERSION']],
                      df_coa [['GL ACCOUNT', 'GL ACCOUNT NAME', 'FINANCIAL STATEMENT', 'GROUP', 'SUB-GROUP', 'DEBITCREDIT']],
                      on='GL ACCOUNT',
                      how='left'
                     )

In [46]:
# We like to change the order of columns a little bit
# This can be done within the pd.merge statement, but for clarity, these two steps have been seperated

data_vlookup=test_vlookup[['COMPANY CODE', 'FISCAL YEAR', 'YEAR-MONTH', 'MONTH', 'COST CENTER', 'GL ACCOUNT', 'GL ACCOUNT NAME', 'VERSION', 'FINANCIAL STATEMENT', 'GROUP', 'SUB-GROUP', 'DEBITCREDIT', 'VALUE']]

In [47]:
# Let´s take a look at the merged data
data_vlookup.head(7)

,COMPANY CODE,FISCAL YEAR,YEAR-MONTH,MONTH,COST CENTER,GL ACCOUNT,GL ACCOUNT NAME,VERSION,FINANCIAL STATEMENT,GROUP,SUB-GROUP,DEBITCREDIT,VALUE
0,ENTITY1,2018,2018-1,1,CCAAA460001,1210,Bonds payable,BUD,Balance sheet,Long-term liabilities,Bonds,Credit,14699.67
1,ENTITY1,2018,2018-1,1,CCAAA460001,2260,Opening inventory,BUD,Income Statement,Cost of sales,Cost of sales,Debit,75719.84
2,ENTITY1,2018,2018-1,1,CCAAA460601,1210,Bonds payable,BUD,Balance sheet,Long-term liabilities,Bonds,Credit,575.44
3,ENTITY1,2018,2018-1,1,CCAAA460601,2260,Opening inventory,BUD,Income Statement,Cost of sales,Cost of sales,Debit,2964.15
4,ENTITY1,2018,2018-1,1,CCAAA601101,2420,Advertising,BUD,Income Statement,Expense,Sales and marketing,Debit,121570.05
5,ENTITY1,2018,2018-1,1,CCAAA601101,2430,Gifts & samples,BUD,Income Statement,Expense,Sales and marketing,Debit,8159.73
6,ENTITY1,2018,2018-1,1,CCAAA601101,2570,Pensions,BUD,Income Statement,Expense,General and administrative,Debit,15698.42


#### 3. Group the new data table

In [48]:
data_grouped = data_vlookup.groupby(
    ['COMPANY CODE',
      'FISCAL YEAR',
      'YEAR-MONTH',
      'MONTH',
      'COST CENTER',
      'GL ACCOUNT',
      'GL ACCOUNT NAME',
      'VERSION',
      'FINANCIAL STATEMENT',
      'GROUP',
      'SUB-GROUP',
      'DEBITCREDIT']).sum()

In [49]:
# Let´s take a look at the grouped table
data_grouped.head(10)

VALUE
COMPANY CODE FISCAL YEAR YEAR-MONTH MONTH COST CENTER GL ACCOUNT GL ACCOUNT NAME                      VERSION FINANCIAL STATEMENT GROUP                 SUB-GROUP                 DEBITCREDIT           
ENTITY1      2018        2018-1     1     CCAAA460001 1210       Bonds payable                        BUD     Balance sheet       Long-term liabilities Bonds                     Credit        14699.67
                                                      2260       Opening inventory                    BUD     Income Statement    Cost of sales         Cost of sales             Debit         75719.84
                                          CCAAA460601 1210       Bonds payable                        BUD     Balance sheet       Long-term liabilities Bonds                     Credit          575.44
                                                      2260       Opening inventory                    BUD     Income Statement    Cost of sales         Cost of sales             Debit          2964.15
                                          CCAAA601101 130        Petty cash account                   BUD     Balance sheet       Current assets        Cash and cash equivalents Debit            70.00
                                                      310        Allowance for doubtful debts account BUD     Balance sheet       Current assets        Accounts receivable       Credit         5548.04
                                                      420        Finished goods                       BUD     Balance sheet       Current assets        Inventory                 Debit          -750.00
                                                      510        Prepayments                          BUD     Balance sheet       Current assets        Other current assets      Debit            41.67
                                                      2420       Advertising                          BUD     Income Statement    Expense               Sales and marketing       Debit        121570.05
                                                      2430       Gifts & samples                      BUD     Income Statement    Expense               Sales and marketing       Debit          8159.73

In [50]:
# The total sum of 'VALUE' in the data file was: 408.405.867,20
# Let´s check, if this amount is correct after merge and group

data_grouped.sum()

VALUE    408405867.2
dtype: float64

#### 4. Export the new data to a csv file

In [51]:
# Write grouped Dataframe to CSV
data_grouped.to_csv(r'DATA_MERGED_GROUPED.csv', sep=';', decimal=',')

#### Additional Step - Pivot the data

In [52]:
data_grouped_pivot = pd.pivot_table(data_grouped, values='VALUE', index=['COMPANY CODE', 'GROUP', 'SUB-GROUP'], columns=['FISCAL YEAR', 'VERSION'], aggfunc=np.sum)

In [53]:
# Let´s take a look at the resulting pivot table
# The order of Groups and Sub-Groups is still not perfect
# To have some variance and sum columns would be useful
# Number formatting would improve readability

data_grouped_pivot

FISCAL YEAR                                                                 2018  \
VERSION                                                                      BUD   
COMPANY CODE GROUP                 SUB-GROUP                                       
ENTITY1      Cost of sales         Cost of sales                      8069232.67   
             Current assets        Accounts receivable                 377765.92   
                                   Cash and cash equivalents         -4691197.08   
                                   Inventory                          2272317.35   
                                   Other current assets                452348.88   
                                   Short term marketable securities   -279933.88   
             Current liabilities   Accounts payable                   3584318.68   
                                   Notes payable                      5992464.48   
                                   Other current liabilities        -10354527.56   
             Equity                Capital                             856554.84   
                                   Retained earnings                   416314.10   
             Expense               Depreciation                        889384.90   
                                   Finance costs                       437620.28   
                                   General and administrative        36109634.10   
                                   Income tax expense                 1448271.84   
                                   Research and development           3081247.81   
                                   Sales and marketing               20961332.22   
             Income                Other Income                       1039136.52   
                                   Revenue                            7127881.50   
             Long term assets      Goodwill                              2400.00   
                                   Intellectual property               394266.84   
                                   Long term marketable securities    1034082.56   
                                   Other long term assets             1388094.05   
                                   Property, plant and equipment     11365931.41   
             Long-term liabilities Bonds                               399839.20   
                                   Mortgages                          7139692.88   

FISCAL YEAR                                                                 2019  \
VERSION                                                                      ACT   
COMPANY CODE GROUP                 SUB-GROUP                                       
ENTITY1      Cost of sales         Cost of sales                      7354128.35   
             Current assets        Accounts receivable                -557063.15   
                                   Cash and cash equivalents         -6277231.93   
                                   Inventory                           821532.76   
                                   Other current assets                560423.23   
                                   Short term marketable securities   -838460.08   
             Current liabilities   Accounts payable                   3266865.24   
                                   Notes payable                      1985183.10   
                                   Other current liabilities         -2379763.83   
             Equity                Capital                            1124792.32   
                                   Retained earnings                   386904.53   
             Expense               Depreciation                       1185653.72   
                                   Finance costs                       166132.68   
                                   General and administrative        32114877.76   
                                   Income tax expense                  489088.06   
                                   Research and dev

In [54]:
# Write pivot data to a csv file
data_grouped_pivot.to_csv(r'DATA_MERGED_GROUPED_PIVOT.csv', sep=';', decimal=',')